### Proximal Policy Optimization

<a href="https://www.youtube.com/watch?v=5P7I-xPq8u8">youtube video om PPO</a>

In [18]:
import gymnasium as gym
from pathlib import Path
from AutoROM import main as autorom
import ale_py

ale = ale_py.ALEInterface()
gym.pprint_registry()

===== classic_control =====
Acrobot-v1                  CartPole-v0                 CartPole-v1
MountainCar-v0              MountainCarContinuous-v0    Pendulum-v1
===== phys2d =====
phys2d/CartPole-v0          phys2d/CartPole-v1          phys2d/Pendulum-v0
===== box2d =====
BipedalWalker-v3            BipedalWalkerHardcore-v3    CarRacing-v3
LunarLander-v3              LunarLanderContinuous-v3
===== toy_text =====
Blackjack-v1                CliffWalking-v1             CliffWalkingSlippery-v1
FrozenLake-v1               FrozenLake8x8-v1            Taxi-v3
===== tabular =====
tabular/Blackjack-v0        tabular/CliffWalking-v0
===== None =====
Ant-v2                      Ant-v3                      GymV21Environment-v0
GymV26Environment-v0        HalfCheetah-v2              HalfCheetah-v3
Hopper-v2                   Hopper-v3                   Humanoid-v2
Humanoid-v3                 HumanoidStandup-v2          InvertedDoublePendulum-v2
InvertedPendulum-v2         Pusher-v2             

In [1]:
import urllib.request

data_root = "https://raw.githubusercontent.com/ageron/data/refs/heads/main"
filename = "ppo_agent_breakout.zip"
urllib.request.urlretrieve(f"{data_root}/{filename}", f"my_{filename}")

('my_ppo_agent_breakout.zip', <http.client.HTTPMessage at 0x7f34c4178f50>)

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

ppo_model = PPO.load("my_ppo_agent_breakout.zip", )
eval_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=1)
eval_stacked = VecFrameStack(eval_env, n_stack=4)
frames = []
obs = eval_stacked.reset()
for _ in range(5000):
    frames.append(eval_stacked.render())
    action, _ = ppo_model.predict(obs)
    obs, reward, done, info = eval_stacked.step(action)
    if done[0]:  # note: there's no `truncated`
        break

eval_stacked.close()

In [19]:
ppo_model.policy

ActorCriticCnnPolicy(
  (features_extractor): NatureCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=3136, out_features=512, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): NatureCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=3136, out_features=512, bias=True)
      (1): ReLU()
    )
  )
  (vf_features_extractor): NatureCNN(
    (cnn): 

In [16]:
import matplotlib.animation
import matplotlib.pyplot as plt

def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,

def plot_animation(frames, repeat=False, interval=40):
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    anim = matplotlib.animation.FuncAnimation(
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()
    return anim

In [17]:
from IPython.display import HTML
anim = plot_animation(frames)

HTML(anim.to_html5_video())


Tiden datorn spelat spelet motsvarar vad ett barn på åttiotalet skulle spelat på ungefär en månad. Påståendet att systemet skulle nå mänsklig nivå på denna tid, och övermänsklig med bara lite mer resurser, är inte reproducerbart. Jag själv spelade detta spel i nintendo-versionen "Arkanoid" på 80-talet som barn och efter en månad var jag långt mycket bättre än denna AI! Första banan, som AI:n inte klarar, går att klara med bara ett fåtal studsar mot padeln -- man öppnar med en gång till vänster och skickar upp bollen så att den missar den nedersta raden men studsar mot väggen och övriga rader. När bollen når ett visst block på översta raden till höger innan man gjort sönder för många block så öppnas den lilla röda boxen i nedre högra hörnet och man kan hoppa vidare till nästa bana. Ett mänskligt barn upptäcker detta jämförelsevis snabbt på egen hand genom experimentering. Det går att styra helt hur bollen studsar med padeln -- kontrollen är en vridratt och en knapp.

<img src="https://static.wikia.nocookie.net/ultimatepopculture/images/c/c3/Vaus.jpg/revision/latest?cb=20190811151518"/>

Med _betydligt_ mycket mer tid än en människa behöver så kan systemet faktiskt nå övermänsklig prestanda, men det spelar fortfarande inte spelet alls som en människa och är fasansfullt ineffektiv i sina rörelser. Med tillräcklig design och stödsystem går det att efterlikna mer mänskliga beteenden -- eller att effektivisera prestandan utan relation till ett särskilt beteende. Detta sista faller oftast inom ramen för _alife_ snarare än _AI_. I de flesta sammanhang defineras intelligens som ett _mänskligt beteende_ snarare än någon inneboende egenskap.

Ett gränsland gällande detta är _autonoma system_ som kan vara mer alife i en del men AI i en annan.